#### train_dcnn_timed
A notebook to analytically evaluate the time it takes to complete the training process for the MMAC-CNN.
- No models are saved
- Reports average and stdev of time to train/validate over `EPOCHS` epochs

In [1]:
import os, time

import torch
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from datasets import PatchNpyDataset
from mmac_net import MMAC_CNN
from mmac_net.train_helpers import loss_acc

import train_mac as train_mmac

/home/cbmolder/anaconda3/envs/medmats/lib/python3.7/site-packages/ipykernel_launcher.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  # Remove the CWD from sys.path while we load stuff.


In [2]:
BATCH_SIZE    = 50
LR_PATIENCE   = 1  # Number of epochs of training (validation?) loss increase before the learning rate clamps down
EPOCHS        = 15 # Number of epochs to train each model
MODEL_SAMPLES = 10 # Sample size of randomly-initialized models to compare
DATA_ROOT     = '' # Root path where patch data/etc. is found.
VERBOSE       = False # Whether to print detailed output during sampling/training

A_path = 'a.npy'

In [3]:
# Optional transforms that normalizes the image for ResNet.
train_tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

val_tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])


# Load the material patch datasets
# - The testing set is only used to track its loss over time,
#   it is not used for parameter tuning / etc.
train_set     = PatchNpyDataset(root = os.path.join(DATA_ROOT, 'patch-set', 'npy', 'train'), transform = train_tf)
train_loader  = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
print(f'Training set   : {len(train_set)} samples')

val_set     = PatchNpyDataset(root = os.path.join(DATA_ROOT, 'patch-set', 'npy', 'val'), transform = val_tf)
val_loader  = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
print(f'Validation set : {len(val_set)} samples')

Training set   : 32188 samples
Validation set : 10724 samples


In [4]:
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
device     = torch.device(device_str)

In [5]:
print(f'Labels   : {train_set.get_labels()}')
print(f'Device   : {device_str}')
print(f'Verbose? : {VERBOSE}')

A = np.load(A_path)
model_times = np.zeros((MODEL_SAMPLES))

for s in range(MODEL_SAMPLES):
    print(f'\n---------------- SAMPLE {s+1}/{MODEL_SAMPLES} ---------------- ')
    
    mmac_cnn  = MMAC_CNN(A, 32).to(device) # Create a new model sample (Second parameter assumes the image patches are 32x32 px)
    optimizer = optim.Adam(mmac_cnn.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=LR_PATIENCE, min_lr=1e-8)
    
    start = time.time()
    for e in range(EPOCHS):
        print(f'~~~~ EPOCH {e+1}/{EPOCHS} ~~~~')
        print(f'lr: {optimizer.param_groups[0]["lr"]}')

        _, _, _ = train_mmac.train(
            mmac_cnn, device, train_loader, optimizer, 
            verbose=VERBOSE
        )

        val_loss, _, _ = train_mmac.validate(
            mmac_cnn, device, val_loader, verbose=VERBOSE
        )
        
        scheduler.step(val_loss)
    
    end = time.time()
    model_times[s] = end - start # Time in seconds
    print('Training time:', end-start,'s')

    # Free CUDA memory
    del mmac_cnn
    del optimizer
    del scheduler

Labels   : ['background', 'tumor', 'bone', 'brain']
Device   : cuda
Verbose? : False

---------------- SAMPLE 1/10 ---------------- 
~~~~ EPOCH 1/15 ~~~~
lr: 0.001



~~~~ EPOCH 2/15 ~~~~
lr: 0.001



~~~~ EPOCH 3/15 ~~~~
lr: 0.001



~~~~ EPOCH 4/15 ~~~~
lr: 0.0001



~~~~ EPOCH 5/15 ~~~~
lr: 0.0001


...


~~~~ EPOCH 12/15 ~~~~
lr: 1.0000000000000004e-08



~~~~ EPOCH 13/15 ~~~~
lr: 1.0000000000000004e-08



~~~~ EPOCH 14/15 ~~~~
lr: 1.0000000000000004e-08



~~~~ EPOCH 15/15 ~~~~
lr: 1.0000000000000004e-08



Training time: 857.2471225261688 s


In [6]:
# After done, print the model times and the mean/stdev
print('Model times to train (seconds):')
print(model_times)

print('\nModel times to train (minutes):')
print(model_times / 60)

print('Mean training time:')
print(np.average(model_times), 's')
print(np.average(model_times/60), 'min')

print('Stdev training time:')
print(np.std(model_times), 's')
print(np.std(model_times/60), 'min')

Model times to train (seconds):
[832.84171605 848.51058888 853.19851255 854.12663126 858.89395547
 859.40023446 859.57188702 861.39385104 865.20594263 857.24712253]

Model times to train (minutes):
[13.88069527 14.14184315 14.21997521 14.23544385 14.31489926 14.32333724
 14.32619812 14.35656418 14.42009904 14.28745204]
Mean training time:
855.0390441894531 s
14.250650736490886 min
Stdev training time:
8.6175063031288 s
0.14362510505214704 min
